# Tutorial:
## Sensitivity Analysis (SA) using the Saltelli sampling scheme with Sobol SA
In this tutorial, we will use the popular Python Sensitivity Analysis Library ([SALib](https://salib.readthedocs.io/en/latest/index.html)) to:
1. Generate a problem set as a dictionary for our Ishigami function that has three inputs
2. Generate 1000 samples for our problem set using the Saltelli<sup>1,2</sup> sampling scheme
3. Execute the Ishigami function for each of our 1000 samples and gather the outputs
4. Compute the sensitivity analysis to generate first-order and total-order sensitivity indices using the Sobol<sup>3</sup> method
5. Interpret the meaning of our results

## Let's get started!
__NOTE__:  Content from this tutorial is taken directly from the SALib ["Basics"](https://salib.readthedocs.io/en/latest/basics.html) walkthrough.

In [1]:
import numpy as np
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami

## Step 1:  Generate the problem dictionary
The Ishigami function has three inputs, 𝑥<sub>1</sub>, 𝑥<sub>2</sub>, 𝑥<sub>3</sub> where 𝑥<sub>𝑖</sub> ∈ [−𝜋, 𝜋].  See the Jupyter notebook tutorial on the [Ishigami function](TBD) for more detailed information.

In [2]:
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359]]
}

## Step 2:  Generate 1000 samples using the Saltelli sampling scheme
Sobol SA requires the use of the Saltelli sampling scheme.  The output of the `saltelli.sample` function is a NumPy array that is the shape 8000 by 3. The Saltelli sampler generated 8000 samples. The Saltelli sampler generates 𝑁∗(2𝐷+2) samples, where in this example N is 1000 (the argument we supplied) and D is 3 (the number of model inputs). The keyword argument calc_second_order=False will exclude second-order indices, resulting in a smaller sample matrix with 𝑁∗(𝐷+2) rows instead.

In [5]:
param_values = saltelli.sample(problem, 1000)
print(f"`param_values` shape:  {param_values.shape}")

`param_values` shape:  (8000, 3)


## Step 3:  Execute the Ishigami function over our sample set
SALib provides a nice wrapper to the Ishigami function that allows the user to directly pass the `param_values` array we just generated into the function directly.

In [6]:
Y = Ishigami.evaluate(param_values)

## Step 4:  Compute first-, second-, and total-order sensitivity indices using the Sobol method
The `sobol.analyze` function will use our problem dictionary and the result of the Ishigami runs (`Y`) to compute first-, second-, and total-order indicies.

In [7]:
Si = sobol.analyze(problem, Y)

Si is a Python dict with the keys "S1", "S2", "ST", "S1_conf", "S2_conf", and "ST_conf". The `_conf` keys store the corresponding confidence intervals, typically with a confidence level of 95%. Use the keyword argument print_to_console=True to print all indices. Or, we can print the individual values from Si as shown below.

## Step 5: Interpret our results
When we execute the following to take a look at our first-order indices (`S1`) for each of our three parameters, we see that 𝑥<sub>1</sub> and 𝑥<sub>2</sub> exibit first-order sensitivities, or the contribution to the output variance by a single parameter, where 𝑥<sub>3</sub> does not.

In [9]:
first_order = Si['S1']
print('First-order:')
print(f"x1: {first_order[0]}, x2: {first_order[1]}, x3: {first_order[2]}")

First-order:
x1: 0.307975488788992, x2: 0.447766610245137, x3: -0.0042545247639770075


Next, we can evaluate the total-order indices to see that they are substantially larger han the first-order indices, which shows the liklihood of higher-order interactions occurring.  Our total-order indices measure the contribution to the output variance caused by a model input, including both its first-order effects (the input varying alone) and all higher-order interactions.

In [10]:
total_order = Si['ST']
print('Total-order:')
print(f"x1: {total_order[0]}, x2: {total_order[1]}, x3: {total_order[2]}")

Total-order:
x1: 0.5601372771365771, x2: 0.4387224992837702, x3: 0.24284474493764008


Finally, we can investigate these higher order interactions by viewing the second-order outputs.  We remember that second-order indicies measure the contribution to the output variance caused by the interaction of two model inputs.

In [11]:
second_order = Si['S2']
print("Second-order:")
print(f"x1-x2:  {second_order[0,1]}")
print(f"x1-x3:  {second_order[0,2]}")
print(f"x2-x3:  {second_order[1,2]}")

Second-order:
x1-x2:  0.012204615255332063
x1-x3:  0.2515257351736557
x2-x3:  -0.009953919522955124


We can see there are strong interactions between 𝑥<sub>1</sub> and 𝑥<sub>3</sub>. Some computing error will appear in the sensitivity indices. For example, we observe a negative value for the 𝑥<sub>2</sub>-𝑥<sub>3</sub> index. Typically, these computing errors shrink as the number of samples increases.

## References
[1] Saltelli, A. (2002). “Making best use of model evaluations to compute sensitivity indices.” Computer Physics Communications, 145(2):280-297, doi:10.1016/S0010-4655(02)00280-1.

[2] Saltelli, A., P. Annoni, I. Azzini, F. Campolongo, M. Ratto, and S. Tarantola (2010). “Variance based sensitivity analysis of model output. Design and estimator for the total sensitivity index.” Computer Physics Communications, 181(2):259-270, doi:10.1016/j.cpc.2009.09.018.

[3] Sobol, I. M. (2001). “Global sensitivity indices for nonlinear mathematical models and their Monte Carlo estimates.” Mathematics and Computers in Simulation, 55(1-3):271-280, doi:10.1016/S0378-4754(00)00270-6.